# Состояние гонки(race condition)

В программе, состоящей из одной горутины, все действия естественным образом упорядочиваются друг за другом: первое действие происходит до второго и т.д. В программе, состоящей из двух или более горутин, действия внутри каждой горутины также упорядочены, но в общем случае мы не знаем произойдёт событие x в одной горутине до, после или одновременно с событием y в другой. Если мы не можем уверенно утверждать, что одно событие произойдёт до другого, то такие события называются **конкурентными**(concurrent).

Некая функция называется **конкурентно-безопасной**(concurrency-safe), если она продолжает правильно работать даже после конкурентного вызова, т.е. вызова из нескольких горутин без использования дополнительных методов синхронизации. Мы можем обобщить это понятие и на множество совместно работающих функций, как методы и операции определённого типа: тип называется конкурентно-безопасным, если все его доступные методы и операции являются конкурентно-безопасными.

Мы можем сделать программу конкурентно-безопасной, не делая конкурентно-безопасным каждый тип программы. На самом деле полностью конкурентно-безопасные типы являются скорее исключением чем правилом. Обращаться к некоторой переменной конкурентно можно только если в документации указано, что так можно делать. Обычно конкурентный доступ ограничивают, заключив переменную в отдельную горутину или поддерживая синхронный доступ с помощью взаимного исключения(mutex'а). Ожидается, что экспортируемые из пакета функции являются конкурентно-безопасными, но это не является строгим правилом.

**Состояние гонки** - это ситуация, когда программа выдаёт неправильный результат из-за чередования выполнения(interleaving) нескольких горутин. Ошибки состояния гонки являются очень опасными, т.к. они могут проявляются только в особых случаях, как например, под большой нагрузкой или при использовании определённого компилятора. Из-за этого их бывает чрезвычайно сложно найти. Можно привести следующий пример состояния гонки:

In [ ]:
import "fmt"

var balance int

func Deposit(amount int) { balance = balance + amount }
func Balance() int       { return balance }

// Alice:
go func() {
    Deposit(200)
    fmt.Println("= ", balance)
}()

// Bob:
go Deposit(100)

В функция Deposit используется "balance + amount" вместо "balance += amount" в демонстрационных целях. Здесь состояние гонки может возникнуть в следующем случае. Допустим, что после запуска горутин, первой была выполнена горутина Alice и произошло вычисление выражения "balance + amount", которое равно 200. После этого была выполнена горутина Bob и баланс стал равен 100. Затем возобновилось выполение горутины Alice, что привело к присваиванию уже вычисленного выражения "balance + amount" переменной balance, т.е. баланс стал равен 200. Получается после двух размещений 100 и 200 баланс стал равен 200, а не 300 как ожидается. Возникло состояние гонки. Этот конкретный вид состояния гонки называется состоянием **гонки данных**(data race). Возникает он когда два или более горутин работают с одной переменной и хотя бы одна из них меняет переменную.

Как уже говорилось выше состояние гонки является сложно уловимой ошибкой. Некоторое разработчики иногда находят оправдания наличию состояния гонки в их коде: "mutex'ы слишком дороги", "от того что потеряется несколько сообщений ничего старшного не случится", "здесь всего лишь происходит логирование". Нужно ясно понять, что **не бывает** доброкачественного состояния гонки. То что ошибка не проявляется на каком-то компиляторе или платформе может придать разработчику ложную уверенность.

Избежать состояния гонки данных можно несколькими способами:
1. Не менять переменную вообще. Т.е. во время инициализации присвоить переменной нужное значение и далее только вычитывать данные из неё. Например, можно заполнить словарь данными при запуске программы, а затем запустить горутины, которые будут только обращаться к словарю, но не менять или добавлять в него значения.
2. Реализовать доступ к переменной только из одной горутины. Другие горутины с помощью каналов уведомляют ограничивающую горутину о намерении получить значение или изменить переменную. Именно это и имеется в виду в мантре Go "do not communicate by sharing memory; instead, share memory by communicating". Горутина, которая выполняет роль посредника для работы с переменной и при этом использует каналы для общения с остальным миром, называется **горутиной-монитором**(monitor goroutine).
3. Разрешить доступ к переменной нескольким горутинам, но так, чтобы в какой-то момент времени только одна горутина работала с переменной. Этот способ реализуется с помощью механизма взаимного исключения и будет рассмотрен отдельно в следующей главе.